Notebook for developing save functionality

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from pof.pof_base import PofBase
from pof.component import Component
pof_base = PofBase()

from pof.helper import get_signature
from pof.pof_container import PofContainer

import pprint
pp = pprint.PrettyPrinter(indent=4)

from pof.paths import Paths

In [ ]:
from pof.data.asset_data import SimpleFleet
from pof.loader.asset_model_loader import AssetModelLoader

sfd = SimpleFleet(Paths().input_path + r"\population_summary.csv")
sfd.load()
sfd.calc_age_forecast(2015, 2024, 2020)

aml = AssetModelLoader(Paths().demo_path + r"\Asset Model - Pole - Timber.xlsx")
comp_data = aml.load(Paths().demo_path + r"\Asset Model - Pole - Timber.xlsx")
comp = Component.from_dict(comp_data["pole"])
comp.fm = {"termites": comp.fm["termites"]}

In [4]:
def get_attr(self, data_req):
    # Get the information needed to create object
    sig_list = list(get_signature(self.__class__))
    
    for attr in sig_list:
        if hasattr(self, attr):
            data_req[attr] = getattr(self, attr)

    return data_req

In [5]:
def unpack_container(data_req):
    for attr, val in data_req.items():
        if isinstance(val, PofContainer):
            data_req[attr] = val.data
        elif isinstance(val, PofBase):
            data_req[attr] = {}
            sig_list = list(get_signature(val.__class__))
            sig_list = [n for n in sig_list if n != "component"]
            for var in sig_list:
                if hasattr(val, var):
                    if getattr(val, var) != NotImplemented:
                        data_req[attr][var] = getattr(val, var)
                    
    for attr, val in data_req.items():
        if isinstance(val, dict):
            for name, value in data_req[attr].items():
                if isinstance(value, (PofContainer, PofBase)):
                    unpack_container(data_req[attr])

    return data_req

In [7]:
def to_dict(self):
    #Get the first layer
    data_req = self.get_attr(data_req = {})
    
    #Add the comp key
    data_req_pole = {}
    data_req_pole["pole"] = data_req

    #Unpack
    data_req_unpacked = unpack_container(data_req_pole)

    # pp.pprint(data_req_unpacked["indicator"])  
    # pp.pprint(data_req)

    return data_req_unpacked

In [8]:
to_dict(comp);

Concept demo

In [10]:
# Class MyClass():

#     data = 10

#     def __init__(self, data = 20)   
#         self.data = data
#         print("created object")

#     def get_data(self):
#         return self.data

#     @classmethod
#     def get_data_class_version(cls):
#         return cls.data


# >>> MyClass.data 10
# >>> MyClass.get_data error
# >>> MyClass.get_dat_class_version 10
# >>> MyClass().data 20
# >>> MyClass().get_data 20

# Method 1 - method

# my_classs = MyClass()
# my_classs_data = mylcass.get_data()

# No - mMyClass.get_data()

# Method 2 - class method
# my_class-data = MyClass.get_data()